# 05 – Experimentos con NN tabular (embeddings + zero-inflated)

En este notebook:
- Evaluamos una arquitectura de red neuronal para predecir `Und_2a_percentage`.
- Validamos el uso de embeddings vs numéricas.
- Comparamos desempeño y analizamos importancia de variables.
- Dejamos documentado el modelo elegido para producción.


In [15]:
from pathlib import Path
import sys

cwd = Path().resolve()
PROJECT_ROOT = None

for parent in [cwd, *cwd.parents]:
    if (parent / "src").is_dir():
        PROJECT_ROOT = parent
        break

if PROJECT_ROOT is None:
    raise RuntimeError("No se encontró carpeta 'src'.")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config.settings import TARGET_COL, RANDOM_STATE, MODELS_DIR, REPORTS_DIR
from src.config.nn_config import NN_MODEL_SUBDIR, NN_KERAS_NAME, NN_PIPELINE_PKL
from src.data.load_data import load_clean_dataset

print("PROJECT_ROOT:", PROJECT_ROOT)
print("TARGET_COL  :", TARGET_COL)


PROJECT_ROOT: D:\Users\dhcertug\OneDrive - Crystal S.A.S\Documentos\HOME\00_PERSONAL\02_CURSOS\PROYECTO\Proyecto_analisis_intermedio_udea
TARGET_COL  : Und_2a_percentage


In [9]:
import pandas as pd
import numpy as np

from src.features.nn_features import reorganize_features_final

df = load_clean_dataset()
y = df[TARGET_COL].values
X = df.drop(columns=[TARGET_COL])

X_clean, embed_cols, num_cols = reorganize_features_final(X)

print("Shape X_clean:", X_clean.shape)
print("Embeddings:", embed_cols)
print("Numéricas :", num_cols)


# Observación:
# En este punto congelamos qué columnas entran al modelo. 
# Cambios posteriores deben justificarse con nueva evidencia (perm importance, negocio, drift)

Eliminando ruido/leakage: ['Rechazo_comp', 'rechazo_flag', 'Tecnologia', 'Tur', 'categoria_producto', 'semana_anio', 'g_art_id']
Variables finales: 16
Shape X_clean: (364832, 19)
Embeddings: ['mp_categoria', 'mp_id', 'Tipo_TEJ', 'planta_id', 'seccion_id', 'producto_id', 'MP', 'maq_id', 'estilo_id', 'C']
Numéricas : ['Col', 'Tal', 'Pas', 'Tal_Fert', 'Col_Fert', 'Componentes']


### Split y preprocesamiento

In [10]:
from sklearn.model_selection import train_test_split
from src.models.nn_preprocessing import preprocess_data

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_clean,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE,
)

train_inputs, test_inputs, encoders, n_nums, scaler = preprocess_data(
    X_train_raw,
    X_test_raw,
    embed_cols,
    num_cols,
)

print("Train inputs keys:", train_inputs.keys())


Train inputs keys: dict_keys(['in_mp_categoria', 'in_mp_id', 'in_Tipo_TEJ', 'in_planta_id', 'in_seccion_id', 'in_producto_id', 'in_MP', 'in_maq_id', 'in_estilo_id', 'in_C', 'in_numerics'])


## Construcción y entrenamiento de la NN

In [11]:
import tensorflow as tf
from tensorflow.keras import callbacks
from src.models.nn_zero_inflated import build_dynamic_model_tuned

model = build_dynamic_model_tuned(
    embed_cols=embed_cols,
    encoders=encoders,
    n_numeric_features=n_nums,
    learning_rate=3e-4,
)

cb = [
    callbacks.EarlyStopping(patience=8, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(patience=4),
]

history = model.fit(
    train_inputs,
    y_train,
    validation_data=(test_inputs, y_test),
    epochs=50,
    batch_size=32,
    callbacks=cb,
    verbose=1,
)

# Observacion:
# “EarlyStopping" y "ReduceLROnPlateau" reducen el riesgo de sobreentrenar y estabilizan el entrenamiento sin necesidad de grid-search manual de épocas.

Epoch 1/50
9121/9121 ━━━━━━━━━━━━━━━━━━━━ 36s 4ms/step - loss: 0.0129 - mae: 0.0700 - rmse: 0.1682 - val_loss: 0.0113 - val_mae: 0.0664 - val_rmse: 0.1575 - learning_rate: 3.0000e-04
Epoch 2/50
9121/9121 ━━━━━━━━━━━━━━━━━━━━ 34s 4ms/step - loss: 0.0114 - mae: 0.0666 - rmse: 0.1579 - val_loss: 0.0110 - val_mae: 0.0670 - val_rmse: 0.1548 - learning_rate: 3.0000e-04
Epoch 3/50
9121/9121 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - loss: 0.0111 - mae: 0.0656 - rmse: 0.1555 - val_loss: 0.0109 - val_mae: 0.0663 - val_rmse: 0.1538 - learning_rate: 3.0000e-04
Epoch 4/50
9121/9121 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 0.0109 - mae: 0.0651 - rmse: 0.1542 - val_loss: 0.0108 - val_mae: 0.0663 - val_rmse: 0.1535 - learning_rate: 3.0000e-04
Epoch 5/50
9121/9121 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - loss: 0.0107 - mae: 0.0646 - rmse: 0.1530 - val_loss: 0.0108 - val_mae: 0.0627 - val_rmse: 0.1535 - learning_rate: 3.0000e-04
Epoch 6/50
9121/9121 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 0.0106 - mae: 0.0639 -

## Evaluación

In [12]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

preds = model.predict(test_inputs).reshape(-1)

mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print(f"R2   : {r2:.4f}")
print(f"MSE  : {mse:.6f}")
print(f"RMSE : {rmse:.6f}")
print(f"MAE  : {mae:.6f}")


2281/2281 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
R2   : 0.7851
MSE  : 0.023394
RMSE : 0.152951
MAE  : 0.064370


In [16]:
# %% [markdown]
# ### Guardar artefactos para dashboard / Streamlit

# %%
import json

# Asegurar carpeta de reports
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# 1) Historia de entrenamiento (para los gráficos)
history_df = pd.DataFrame(history.history)
hist_path = REPORTS_DIR / "history_nn.csv"   # <--- el nombre que espera la app
history_df.to_csv(hist_path, index_label="epoch")
print("✅ History guardado en:", hist_path)

# 2) Métricas agregadas (para las tarjetas de la app)
metrics = {
    "model_name": "nn_zero_inflated_experiments",
    "R2": float(r2),
    "MSE": float(mse),
    "RMSE": float(rmse),
    "MAE": float(mae),
}
metrics_path = REPORTS_DIR / "metrics_nn.json"
metrics_path.write_text(json.dumps(metrics, indent=2), encoding="utf-8")
print("✅ Métricas guardadas en:", metrics_path)

# 3) Hyperparámetros básicos del experimento
hyperparams = {
    "learning_rate": 3e-4,
    "epochs": 50,
    "batch_size": 32,
    "patience_es": 8,
    "patience_lr": 4,
    "test_size": 0.2,
    "random_state": RANDOM_STATE,
}
hyper_path = REPORTS_DIR / "hyperparams_nn.json"
hyper_path.write_text(json.dumps(hyperparams, indent=2), encoding="utf-8")
print("✅ Hyperparámetros guardados en:", hyper_path)

# 4) Tabla de comparación de modelos
comp_path = REPORTS_DIR / "model_comparison.csv"
row = {"model": "nn_zero_inflated_experiments", "R2": r2, "RMSE": rmse}

if comp_path.exists():
    comp_df = pd.read_csv(comp_path)
    comp_df = pd.concat([comp_df, pd.DataFrame([row])], ignore_index=True)
else:
    comp_df = pd.DataFrame([row])

comp_df.to_csv(comp_path, index=False)
print("✅ Model comparison actualizado en:", comp_path)


✅ History guardado en: D:\Users\dhcertug\OneDrive - Crystal S.A.S\Documentos\HOME\00_PERSONAL\02_CURSOS\PROYECTO\Proyecto_analisis_intermedio_udea\src\reports\history_nn.csv
✅ Métricas guardadas en: D:\Users\dhcertug\OneDrive - Crystal S.A.S\Documentos\HOME\00_PERSONAL\02_CURSOS\PROYECTO\Proyecto_analisis_intermedio_udea\src\reports\metrics_nn.json
✅ Hyperparámetros guardados en: D:\Users\dhcertug\OneDrive - Crystal S.A.S\Documentos\HOME\00_PERSONAL\02_CURSOS\PROYECTO\Proyecto_analisis_intermedio_udea\src\reports\hyperparams_nn.json
✅ Model comparison actualizado en: D:\Users\dhcertug\OneDrive - Crystal S.A.S\Documentos\HOME\00_PERSONAL\02_CURSOS\PROYECTO\Proyecto_analisis_intermedio_udea\src\reports\model_comparison.csv


In [13]:
import joblib

model_dir = MODELS_DIR / NN_MODEL_SUBDIR
model_dir.mkdir(parents=True, exist_ok=True)

keras_path = model_dir / NN_KERAS_NAME
pipe_path = model_dir / NN_PIPELINE_PKL

model.save(keras_path)

pipeline_artefactos = {
    "keras_model_path": keras_path,
    "encoders": encoders,
    "scaler": scaler,
    "embed_cols": embed_cols,
    "num_cols": num_cols,
}

joblib.dump(pipeline_artefactos, pipe_path)

print("Modelo guardado en:", keras_path)
print("Pipeline guardado en:", pipe_path)


Modelo guardado en: D:\Users\dhcertug\OneDrive - Crystal S.A.S\Documentos\HOME\00_PERSONAL\02_CURSOS\PROYECTO\Proyecto_analisis_intermedio_udea\src\data\models\nn_zero_inflated\modelo_defectos.keras
Pipeline guardado en: D:\Users\dhcertug\OneDrive - Crystal S.A.S\Documentos\HOME\00_PERSONAL\02_CURSOS\PROYECTO\Proyecto_analisis_intermedio_udea\src\data\models\nn_zero_inflated\pipeline_completo.pkl


### Importancia por permutación

In [14]:
from src.models.importance import calculate_permutation_importance

# Métricas de Keras: [loss, mae, rmse] -> rmse está en índice 2
RMSE_INDEX = 2

imps = calculate_permutation_importance(
    model=model,
    X_dict=test_inputs,
    y_true=y_test,
    metric_index=RMSE_INDEX,
    sample_size=10000,
)

sorted_imps = sorted(imps.items(), key=lambda x: x[1], reverse=True)

for name, imp in sorted_imps[:10]:
    print(f"{name}: ΔRMSE = {imp:.6f}")
    

# Observacion:
# Permutation importance a nivel de input dict permite validar si la arquitectura está usando las señales correctas y justificar exclusiones futuras. 
# No es una explicación local estilo SHAP, pero es suficientemente estable para decisiones de feature selection.


in_seccion_id: ΔRMSE = 0.266891
in_numerics: ΔRMSE = 0.056336
in_producto_id: ΔRMSE = 0.013092
in_mp_id: ΔRMSE = 0.010722
in_maq_id: ΔRMSE = 0.000948
in_estilo_id: ΔRMSE = 0.000229
in_Tipo_TEJ: ΔRMSE = 0.000079
in_mp_categoria: ΔRMSE = 0.000061
in_MP: ΔRMSE = 0.000012
in_planta_id: ΔRMSE = -0.000003
